In [1]:
!pip install markovify

     |████████████████████████████████| 245kB 4.9MB/s 
  Created wheel for markovify: filename=markovify-0.8.2-cp36-none-any.whl size=18261 sha256=18c60250f5d17b8ad93b1138f5ca520767ffbea93180f87e7f6dba1a08411b71
  Stored in directory: /root/.cache/pip/wheels/f4/3f/cc/c2750c71a820928e12f9609ff3a99d7b2c0d93eb61b7170189
Successfully built markovify


In [2]:
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

import markovify

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks"

'Copy of 04_02_inferential_statistics_assign.ipynb'
'Copy of Practice - Python for data scientists II - accessing data - rest.ipynb'
'Copy of Practice - Python for data scientists II - accessing data - scraping.ipynb'
'Copy of Practice - Python for data scientists III - eda viz.ipynb'
'Deep Learning'
'Natural Language Processing'


##### Goals
* Predict movie genre from conversation
* Generate dialogue
* Chatbot

In [ ]:
with open("/content/drive/My Drive/Colab Notebooks/Natural Language Processing/18754_24465_bundle_archive/movie_lines.txt", encoding="latin-1") as f:
    lines = f.readlines()

movie_lines = []

for elem in lines:
    line_list = elem.strip("\n").split(" +++$+++ ")
    movie_lines.append(line_list)

movie_lines

In [ ]:
with open(
    "/content/drive/My Drive/Colab Notebooks/Natural Language Processing/18754_24465_bundle_archive/movie_conversations.txt", encoding="latin-1"
) as f:
    lines = f.readlines()

movie_conversations = []

for elem in lines:
    line_list = elem.strip("\n").split(" +++$+++ ")
    line_list[-1] = line_list[-1].strip("][").split(", ")
    movie_conversations.append(line_list)

    genres = []
    for i in line_list[-1]:
        i = i.strip("'")
        genres.append(i)
    line_list[-1] = genres

movie_conversations

In [ ]:
with open(
    "/content/drive/My Drive/Colab Notebooks/Natural Language Processing/18754_24465_bundle_archive/movie_characters_metadata.txt", encoding="latin-1"
) as f:
    lines = f.readlines()

characters = []

for elem in lines:
    line_list = elem.strip("\n").split(" +++$+++ ")
    characters.append(line_list)

characters = characters[1:]
characters

In [ ]:
with open(
    "/content/drive/My Drive/Colab Notebooks/Natural Language Processing/18754_24465_bundle_archive/movie_titles_metadata.txt", encoding="latin-1"
) as f:
    lines = f.readlines()

movie_titles = []

for elem in lines:
    line_list = elem.strip("\n").split(" +++$+++ ")
    line_list[-1] = line_list[-1].strip("][").split(", ")

    genres = []
    for i in line_list[-1]:
        i = i.strip("'")
        genres.append(i)
    line_list[-1] = genres

    movie_titles.append(line_list)

movie_titles

In [10]:
import sys

In [12]:
%%time


movie_d = dict()
for movie_title in movie_titles:
    small_d = dict()
    for conversation in movie_conversations:
        if movie_title[0] == conversation[2]:
            for line in movie_lines:
                if line[0] in conversation[-1]:
                    line_list.append(line[-1])
    
    small_d['title'] = movie_title[1]
    small_d['genres'] = movie_title[-1]
    small_d['lines'] = line_list
    movie_d[movie_title[0]] = small_d
    
    print(movie_d)
    sys.exit()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
movie_d

In [ ]:
genre_list = []
for k,v in movie_d.items():
  for movie in movie_titles:
    if movie[0] == k:
      genre_list.append(" ".join(movie[-1]))
# for movie_title in movie_titles:
#     genre = " ".join(movie_title[-1])
#     genre_list.append(genre)

In [ ]:
genre_list

In [ ]:
all_dialogue = []
for k,v in movie_d.items():
  all_dialogue.append(" ".join(v))

In [ ]:
tokenized_list = []
for dialogue in all_dialogue:
    tokenized = sent_tokenize(dialogue)
    tokenized_list.append(tokenized)

tokenized_list

In [ ]:
for dialogue in tokenized_list:
    model = markovify.Text(dialogue, state_size=3)
    for i in range(5):
        print(model.make_short_sentence(max_chars=200, min_chars=30, tries=100), "\n")
    print("`" * 100, "\n")

In [ ]:
def preprocess(docs):
    lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer("english")
    preprocessed = []

    for doc in docs:
        tokenized = word_tokenize(doc)

        cleaned = [
            stemmer.stem(lemmatizer.lemmatize(token.lower()))
            for token in tokenized
            if not token.lower() in stopwords.words("english")
            if token.isalpha()
        ]

        untokenized = " ".join(cleaned)
        preprocessed.append(untokenized)

    return preprocessed

In [ ]:
preprocessed = preprocess(all_dialogue)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(preprocessed, genre_list, test_size=0.3, random_state=33)

In [ ]:
%%time

model = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(n_estimators=100)),
])

model.fit(x_train, y_train)

CPU times: user 5.27 s, sys: 64 ms, total: 5.33 s
Wall time: 5.35 s


In [ ]:
predictions = model.predict(x_test)
print(classification_report(y_test, predictions))

In [ ]:
scores = cross_val_score(model, preprocessed, genre_list, 
                         cv=10, scoring='f1_macro')

print(scores.mean())
print(scores.std())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


0.02540243197310743
0.009590448582636583


In [ ]:
y_pred = model.predict(x_test)
y_true = y_test

In [ ]:
y_pred

In [ ]:
y_true